In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/Cleaned_Manufacturing_Data (1).xlsx'
import pandas as pd
df = pd.read_excel(file_path)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df.head()

,Batch,Downtime factor,Downtime minutes,Downtime factors.Description,Operator Error,Date,Product,Operator,Start Time,End Time,Minutes Of Production,Flavor,Product_Size,Min batch time
0,422111,2,60,Batch change,Yes,2024-08-29,OR-600,Mac,11:50:00,14:05:00,135.0,Orange,600 ml,60
1,422111,7,15,Machine failure,No,2024-08-29,OR-600,Mac,11:50:00,14:05:00,135.0,Orange,600 ml,60
2,422112,2,20,Batch change,Yes,2024-08-29,LE-600,Mac,14:05:00,15:45:00,100.0,Lemon lime,600 ml,60
3,422112,8,20,Batch coding error,Yes,2024-08-29,LE-600,Mac,14:05:00,15:45:00,100.0,Lemon lime,600 ml,60
4,422113,2,50,Batch change,Yes,2024-08-29,LE-600,Mac,15:45:00,17:35:00,110.0,Lemon lime,600 ml,60


## ***1. Line Productivity:***

## **• How many batches met the optimum production time?**


In [ ]:
# Get unique batches and their production time vs min batch time
batch_stats = df.groupby('Batch').agg({
    'Minutes Of Production': 'first',  # Production time is same for all records of a batch
    'Min batch time': 'first',        # Min time is same for all records of a batch
    'Product': 'first',              # Include product name for reference
    'Flavor': 'first'                # Include flavor for reference
}).reset_index()

# Calculate which batches were faster than optimum time (strictly lower)
batch_stats['Faster_Than_Optimum'] = batch_stats['Minutes Of Production'] < batch_stats['Min batch time']

# Count the results
total_batches = len(batch_stats)
faster_batches = batch_stats['Faster_Than_Optimum'].sum()
percentage_faster = (faster_batches / total_batches) * 100

# Get the list of batches that were faster than optimum
faster_batches_list = batch_stats[batch_stats['Faster_Than_Optimum']][
    ['Batch', 'Product', 'Flavor', 'Minutes Of Production', 'Min batch time']
]

# Print results
print(f"Total batches in dataset: {total_batches}")
print(f"Batches with production time <= minimum batch time: {faster_batches} ({percentage_faster:.2f}%)")
print(faster_batches_list.to_string(index=False))

Total batches in dataset: 38
Batches with production time <= minimum batch time: 2 (5.26%)
 Batch Product     Flavor  Minutes Of Production  Min batch time
422116  LE-600 Lemon lime                   60.0              60
422136  DC-600  Diet Cola                   60.0              60


## **• Batch Production Efficiency**



In [ ]:
# Convert 'No downtime' to 0 in Downtime minutes
df['Downtime minutes'] = pd.to_numeric(df['Downtime minutes'], errors='coerce').fillna(0)

In [ ]:
# Group by Batch to aggregate multiple records
batch_stats = df.groupby('Batch').agg({
    'Minutes Of Production': 'first',  # Clock time (same for all records of a batch)
    'Downtime minutes': 'sum',         # Sum all downtime events
    'Operator': 'first',               # Batch operator
    'Product': 'first'                 # Batch product
}).reset_index()

# Calculate productive time and efficiency
batch_stats['Productive Time'] = batch_stats['Minutes Of Production'] - batch_stats['Downtime minutes']
batch_stats['Efficiency'] = (batch_stats['Productive Time'] / batch_stats['Minutes Of Production'] * 100).round(1)

# Convert to hours:minutes
def mins_to_hhmm(mins):
    return f"{int(mins//60)}h {int(mins%60)}m"

batch_stats['Clock_HHMM'] = batch_stats['Minutes Of Production'].apply(mins_to_hhmm)
batch_stats['Downtime_HHMM'] = batch_stats['Downtime minutes'].apply(mins_to_hhmm)

# Sort by worst efficiency first
batch_stats = batch_stats.sort_values('Efficiency', ascending=True)

# Final output
print(batch_stats[[
    'Batch', 'Product', 'Operator',
    'Minutes Of Production', 'Clock_HHMM',
    'Downtime minutes', 'Downtime_HHMM',
    'Efficiency'
]].to_string(index=False))

 Batch Product Operator  Minutes Of Production Clock_HHMM  Downtime minutes Downtime_HHMM  Efficiency
422111  OR-600      Mac                  135.0     2h 15m                75        1h 15m        44.4
422123  CO-600   Dennis                  133.0     2h 13m                73        1h 13m        45.1
422147   CO-2L  Charlie                  205.0     3h 25m               107        1h 47m        47.8
422140  RB-600      Dee                  123.0      2h 3m                63         1h 3m        48.8
422118  CO-600      Dee                  120.0      2h 0m                60         1h 0m        50.0
422143  RB-600   Dennis                  118.0     1h 57m                58        0h 58m        50.8
422128  CO-600  Charlie                  112.0     1h 52m                52        0h 52m        53.6
422120  CO-600      Dee                  112.0     1h 52m                52        0h 52m        53.6
422113  LE-600      Mac                  110.0     1h 49m                50       

## ***2. Products Analysis:***

## **•	What is the most frequently produced product?**

In [ ]:
# Calculate production metrics
production_stats = df.groupby(['Product', 'Flavor']).agg(
    Batch_Count=('Batch', 'nunique'),
    Total_Production_Minutes=('Minutes Of Production', 'sum')
).reset_index()

# Convert to hours and calculate all metrics
production_stats['Total_Production_Hours'] = (production_stats['Total_Production_Minutes'] / 60).round(1)
production_stats['Hours_per_Batch'] = (production_stats['Total_Production_Minutes'] / production_stats['Batch_Count'] / 60).round(2)
production_stats['Mins_per_Batch'] = (production_stats['Total_Production_Minutes'] / production_stats['Batch_Count']).round(1)

# Calculate percentages
total_production_hours = production_stats['Total_Production_Hours'].sum()
production_stats['Percentage_of_Total_Products'] = (production_stats['Total_Production_Hours'] / total_production_hours * 100).round(1)

# Sort by production volume
production_stats = production_stats.sort_values('Total_Production_Hours', ascending=False)

# Display results
print("Production Analysis by Product:")
print(production_stats[['Product', 'Flavor', 'Batch_Count',
                       'Total_Production_Hours', 'Hours_per_Batch',
                       'Mins_per_Batch', 'Percentage_of_Total_Products']].to_string(index=False))

Production Analysis by Product:
Product     Flavor  Batch_Count  Total_Production_Hours  Hours_per_Batch  Mins_per_Batch  Percentage_of_Total_Products
 CO-600       Cola           15                    41.6             2.77           166.2                          35.5
  CO-2L       Cola            5                    29.7             5.93           355.8                          25.3
 RB-600 Root Berry            7                    18.6             2.66           159.9                          15.9
 LE-600 Lemon lime            6                    13.4             2.23           134.0                          11.4
 DC-600  Diet Cola            4                     9.5             2.37           142.5                           8.1
 OR-600     Orange            1                     4.5             4.50           270.0                           3.8


## **• Which product has the highest/lowest production efficiency?**

In [ ]:
# Calculate total downtime per batch (summing all downtime events)
batch_downtime = df.groupby('Batch').agg(
    Total_Downtime=('Downtime minutes', 'sum'),
    Downtime_Events=('Downtime minutes', 'count')
).reset_index()

# Get production time (first record per batch)
batch_production = df.groupby('Batch').first()[['Product', 'Flavor', 'Product_Size', 'Minutes Of Production']]

# Merge the data
batch_stats = pd.merge(batch_production, batch_downtime, on='Batch')

# Calculate true efficiency and downtime percentage
batch_stats['Productive_Minutes'] = batch_stats['Minutes Of Production'] - batch_stats['Total_Downtime']
batch_stats['Efficiency'] = (batch_stats['Productive_Minutes'] / batch_stats['Minutes Of Production'])
batch_stats['Downtime_Percentage'] = (batch_stats['Total_Downtime'] / batch_stats['Minutes Of Production'])

# Aggregate by product and format
product_stats = batch_stats.groupby(['Product', 'Flavor', 'Product_Size']).agg({
    'Efficiency': 'mean',
    'Downtime_Percentage': 'mean',
    'Minutes Of Production': 'sum',
    'Productive_Minutes': 'sum',
    'Total_Downtime': 'sum',
    'Batch': 'count'
}).rename(columns={
    'Batch': 'Batch_Count',
    'Minutes Of Production': 'Total_Clock_Minutes',
    'Total_Downtime': 'Total_Downtime_Minutes'
})

# Convert to hours and format percentages
product_stats['Total_Production_Hours'] = (product_stats['Total_Clock_Minutes'] / 60).round(1)
product_stats['Productive_Hours'] = (product_stats['Productive_Minutes'] / 60).round(1)
product_stats['Downtime_Hours'] = (product_stats['Total_Downtime_Minutes'] / 60).round(1)
product_stats['Efficiency'] = (product_stats['Efficiency'] * 100).round(1).astype(str) + '%'
product_stats['Downtime_Percentage'] = (product_stats['Downtime_Percentage'] * 100).round(1).astype(str) + '%'

# Final formatted output
result = product_stats[['Batch_Count', 'Efficiency', 'Downtime_Percentage',
                       'Total_Production_Hours', 'Productive_Hours', 'Downtime_Hours']]\
          .sort_values('Efficiency', ascending=False)

print(result.to_string())

                                 Batch_Count Efficiency Downtime_Percentage  Total_Production_Hours  Productive_Hours  Downtime_Hours
Product Flavor     Product_Size                                                                                                      
DC-600  Diet Cola  600 ml                  4      71.7%               28.3%                     5.9               4.0             1.9
LE-600  Lemon lime 600 ml                  6      71.0%               29.0%                     8.8               6.0             2.8
CO-600  Cola       600 ml                 15      67.3%               32.7%                    23.2              15.0             8.2
CO-2L   Cola       2 L                     5      66.1%               33.9%                    12.8               8.2             4.6
RB-600  Root Berry 600 ml                  7      64.4%               35.6%                    11.3               7.0             4.3
OR-600  Orange     600 ml                  1      44.4%       

In [ ]:
# Convert Start Time to datetime and extract hour
df['Start Time'] = pd.to_datetime(df['Start Time'])
df['Hour'] = df['Start Time'].dt.hour

# Define shifts
def categorize_shift(hour):
    if 6 <= hour < 14: return 'Morning (6AM-2PM)'
    elif 14 <= hour < 22: return 'Afternoon (2PM-10PM)'
    else: return 'Night (10PM-6AM)'

# Apply shift categorization
df['Shift'] = df['Hour'].apply(categorize_shift)

# Define custom sorting order for shifts
shift_order = ['Morning (6AM-2PM)', 'Afternoon (2PM-10PM)', 'Night (10PM-6AM)']

# Get unique batches per shift and date
batches_per_shift = df.groupby(['Shift', 'Date'])['Batch'].nunique().reset_index()

# Calculate average batches per shift (overall), round, and reindex
overall_avg = (batches_per_shift.groupby('Shift')['Batch']
               .mean()
               .round(1)
               .reindex(shift_order))

# Get unique batches per shift, product, and date
batches_per_shift_product = df.groupby(['Shift', 'Product', 'Date'])['Batch'].nunique().reset_index()

# Calculate average batches per shift by product, fill NaN with 0, round, and reindex
product_avg = (batches_per_shift_product.groupby(['Shift', 'Product'])['Batch']
               .mean()
               .unstack()
               .fillna(0)
               .round(1)
               .reindex(shift_order))

print("Average number of batches produced per shift (overall):")
print(overall_avg)
print("\nAverage number of batches produced per shift by product:")
print(product_avg)

Average number of batches produced per shift (overall):
Shift
Morning (6AM-2PM)       3.8
Afternoon (2PM-10PM)    4.0
Night (10PM-6AM)        2.3
Name: Batch, dtype: float64

Average number of batches produced per shift by product:
Product               CO-2L  CO-600  DC-600  LE-600  OR-600  RB-600
Shift                                                              
Morning (6AM-2PM)       1.0     4.0     2.0     0.0     1.0     3.0
Afternoon (2PM-10PM)    3.0     5.0     2.0     6.0     0.0     0.0
Night (10PM-6AM)        1.0     2.0     0.0     0.0     0.0     4.0


<ipython-input-10-88e6847fb527>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start Time'] = pd.to_datetime(df['Start Time'])


## ***3. Downtime Factors:***

## **• What are the most common causes of downtime?**



In [ ]:
downtime_data = df[df['Downtime minutes'] > 0].copy()

# Count all individual records per cause
dt_cause_analysis = downtime_data.groupby('Downtime factors.Description').agg(
    Occurrences=('Downtime minutes', 'count'),  # Counts ALL records
    Total_Minutes=('Downtime minutes', 'sum'),
    Affected_Batches=('Batch', 'nunique')       # Unique batches per cause
).sort_values('Total_Minutes', ascending=False)

# Convert to hours and format
dt_cause_analysis['Total_Hours'] = (dt_cause_analysis['Total_Minutes'] / 60).round(1)
dt_cause_analysis['Avg_Minutes_Per_Event'] = (dt_cause_analysis['Total_Minutes'] / dt_cause_analysis['Occurrences']).round(1)

# Rename columns for clarity
dt_cause_analysis = dt_cause_analysis.rename(columns={
    'Downtime factors.Description': 'Cause',
    'Occurrences': 'Total_Events',
    'Total_Minutes': 'Total_Minutes'
})

# Display full results
print("\nDowntime Cause Analysis:")
print(dt_cause_analysis[['Total_Events', 'Affected_Batches', 'Total_Minutes',
                        'Total_Hours', 'Avg_Minutes_Per_Event']].to_string())


Downtime Cause Analysis:
                              Total_Events  Affected_Batches  Total_Minutes  Total_Hours  Avg_Minutes_Per_Event
Downtime factors.Description                                                                                   
Machine adjustment                      12                12            332          5.5                   27.7
Machine failure                         11                11            254          4.2                   23.1
Inventory shortage                       9                 9            225          3.8                   25.0
Batch change                             5                 5            160          2.7                   32.0
Batch coding error                       6                 6            145          2.4                   24.2
Other                                    6                 6             74          1.2                   12.3
Product spill                            3                 3             57   

## **•	How often do operator errors contribute to downtime?**

In [ ]:
# Filter valid downtime events
downtime_events = df[df['Downtime minutes'] > 0].copy()

# Calculate total downtime
total_downtime_minutes = downtime_events['Downtime minutes'].sum()

# Group by operator error status
operator_error_stats = downtime_events.groupby('Operator Error').agg(
    Total_Events=('Downtime minutes', 'count'),
    Total_Minutes=('Downtime minutes', 'sum')
).reset_index()

# Calculate percentages and hours
operator_error_stats['Total_Hours'] = (operator_error_stats['Total_Minutes'] / 60).round(1)
operator_error_stats['Percentage_of_Total_Events'] = (operator_error_stats['Total_Events'] / 61 * 100).round(1)

# Format output
result = operator_error_stats[['Operator Error', 'Total_Events', 'Total_Minutes', 'Total_Hours', 'Percentage_of_Total_Events']]
print(result.to_string(index=False))

Operator Error  Total_Events  Total_Minutes  Total_Hours  Percentage_of_Total_Events
            No            29            612         10.2                        47.5
           Yes            32            776         12.9                        52.5


## **•	Most common downtime factor caused by operator error**

In [ ]:
# Get operator-caused downtime
operator_errors = df[(df['Downtime minutes'] > 0) &
                    (df['Operator Error'] == 'Yes')]

# Analyze and format results
error_analysis = (
    operator_errors.groupby('Downtime factors.Description')
    .agg(Occurrences=('Downtime minutes', 'count'),
         Total_Downtime_Hours=('Downtime minutes', lambda x: round(x.sum()/60, 1))
    )
    .sort_values('Total_Downtime_Hours', ascending=False)
    .reset_index()
)

# Rename for clarity
error_analysis.columns = ['Error Type', 'Occurrences', 'Total Hours']
print(error_analysis.to_string(index=False))

        Error Type  Occurrences  Total Hours
Machine adjustment           12          5.5
      Batch change            5          2.7
Batch coding error            6          2.4
     Product spill            3          1.0
 Calibration error            3          0.8
      Label switch            3          0.6


## **• Are there patterns in downtime based on time of day or shift?**

In [ ]:
df['Start Hour'] = pd.to_datetime(df['Start Time'], format='%H:%M:%S').dt.hour

df['Shift'] = df['Start Hour'].apply(categorize_shift)

# Analyze downtime patterns
downtime_patterns = df[df['Downtime minutes'] > 0].groupby(['Shift', 'Start Hour']).agg(
    Total_Downtime=('Downtime minutes', 'sum'),
    Occurrences=('Downtime minutes', 'count')
).reset_index()

# Calculate hourly averages
hourly_avg = downtime_patterns.groupby('Start Hour').agg(
    Avg_Downtime=('Total_Downtime', 'mean')
).reset_index()
# Shift-wise summary
shift_summary = df[df['Downtime minutes'] > 0].groupby('Shift').agg(
    Total_Downtime_Minutes=('Downtime minutes', 'sum'),
    Occurrences=('Downtime minutes', 'count')
).sort_values('Total_Downtime_Minutes', ascending=False)

shift_summary['Total_Hours'] = (shift_summary['Total_Downtime_Minutes'] / 60).round(1)
print("\nShift-wise Downtime Summary:")
print(shift_summary[['Occurrences', 'Total_Downtime_Minutes', 'Total_Hours']])


Shift-wise Downtime Summary:
                      Occurrences  Total_Downtime_Minutes  Total_Hours
Shift                                                                 
Afternoon (2PM-10PM)           25                     584          9.7
Morning (6AM-2PM)              22                     534          8.9
Night (10PM-6AM)               14                     270          4.5


## ***4. Operator Performance:***

## **• Which operator has the best/worst efficiency ratio?**

In [ ]:
# Step 1: Aggregate data by Batch-Operator to handle multiple records
batch_operator_stats = df.groupby(['Batch', 'Operator']).agg({
    'Minutes Of Production': 'max',  # Use max to get the total production time (including downtime)
    'Downtime minutes': 'sum'       # Sum all downtime minutes for the batch
}).reset_index()

# Step 2: Calculate actual production time (subtract downtime from total production time)
batch_operator_stats['Actual Production Minutes'] = (
    batch_operator_stats['Minutes Of Production'] -
    batch_operator_stats['Downtime minutes']
)

# Step 3: Aggregate by Operator
operator_stats = batch_operator_stats.groupby('Operator').agg(
    total_actual_production=('Actual Production Minutes', 'sum'),
    total_downtime=('Downtime minutes', 'sum')
).reset_index()

# Calculate efficiency ratio (production/downtime)
operator_stats['Efficiency Ratio'] = (
    operator_stats['total_actual_production'] /
    operator_stats['total_downtime'].replace(0, 0.1)  # Avoid division by zero
)

# Sort by efficiency (best to worst)
operator_stats = operator_stats.sort_values('Efficiency Ratio', ascending=False)

print(operator_stats[['Operator', 'Efficiency Ratio',
                     'total_actual_production', 'total_downtime']]
      .to_string(index=False, float_format='%.1f'))

# Identify best and worst
best_op = operator_stats.iloc[0]
worst_op = operator_stats.iloc[-1]

print(f"\nMost Efficient Operator: {best_op['Operator']} (Ratio: {best_op['Efficiency Ratio']:.1f})")
print(f"Least Efficient Operator: {worst_op['Operator']} (Ratio: {worst_op['Efficiency Ratio']:.1f})")

Operator  Efficiency Ratio  total_actual_production  total_downtime
 Charlie               2.0                    774.0             384
     Dee               1.8                    660.0             370
  Dennis               1.7                    518.0             302
     Mac               1.6                    518.0             332

Most Efficient Operator: Charlie (Ratio: 2.0)
Least Efficient Operator: Mac (Ratio: 1.6)


In [19]:
# Step 1: Group by Batch-Operator-Shift to aggregate all downtime factors
batch_operator_shift_stats = df.groupby(['Batch', 'Operator', 'Shift']).agg(
    Total_Production_With_Downtime=('Minutes Of Production', 'max'),  # Max to get full duration
    Total_Downtime=('Downtime minutes', 'sum')                       # Sum all downtime factors
).reset_index()

# Step 2: Calculate actual production time (excluding downtime)
batch_operator_shift_stats['Actual_Production'] = (
    batch_operator_shift_stats['Total_Production_With_Downtime'] -
    batch_operator_shift_stats['Total_Downtime']
)

# Step 3: Aggregate by Operator-Shift
operator_shift_stats = batch_operator_shift_stats.groupby(['Operator', 'Shift']).agg(
    Total_Actual_Production=('Actual_Production', 'sum'),
    Total_Downtime=('Total_Downtime', 'sum')
).reset_index()

# Calculate efficiency ratio (production/downtime)
operator_shift_stats['Efficiency'] = (
    operator_shift_stats['Total_Actual_Production'] /
    operator_shift_stats['Total_Downtime'].replace(0, 0.001)  # Avoid division by zero
).round(1)

# Convert Shift to categorical with specified order
shift_order = ['Morning (6AM-2PM)', 'Afternoon (2PM-10PM)', 'Night (10PM-6AM)']
operator_shift_stats['Shift'] = pd.Categorical(
    operator_shift_stats['Shift'],
    categories=shift_order,
    ordered=True
)

# Create pivot table with ordered columns
pivot_result = operator_shift_stats.pivot(
    index='Operator',
    columns='Shift',
    values='Efficiency'
).fillna(0)[shift_order]  # Explicit column ordering

# Format output
styled_result = pivot_result.style\
    .background_gradient(cmap='RdYlGn', axis=None, vmin=0, vmax=5)\
    .format('{:.1f}')\
    .set_caption('Operator Efficiency Ratio (Actual Production/Downtime) by Shift')\
    .highlight_null('white')

display(styled_result)

Shift,Morning (6AM-2PM),Afternoon (2PM-10PM),Night (10PM-6AM)
Operator,,,
Charlie,0.0,1.9,4.0
Dee,2.4,0.0,1.3
Dennis,1.7,1.5,0.0
Mac,1.0,1.7,3.1
